## Neural Network Optimizatio

---

## Overview

This project aims to enhance neural network efficiency for resource-constrained devices through the combined application of pruning, quantization, and weight sharing, integrated with Dispersive Flies Optimization (DFO). The goal is to reduce the model's size and computational requirements while maintaining high accuracy, making it suitable for deployment on devices such as IoT devices, microcontrollers, and smartphones.

## Table of Contents

- [Overview](#overview)
- [Prerequisites](#prerequisites)
- [Installation](#installation)
- [Usage](#usage)
  - [Initial Training](#initial-training)
  - [Pruning](#pruning)
  - [Quantization](#quantization)
  - [Weight Sharing](#weight-sharing)
  - [Evaluation](#evaluation)
- [Contributing](#contributing)
- [License](#license)
- [Contact](#contact)

## Prerequisites

Before we begin, ensure we have met the following requirements:

- Python 3.6 or higher
- PyTorch
- Scikit-learn
- NumPy


In [ ]:
## Installation
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# Set a random seed for reproducibility
np.random.seed(42)

In [ ]:
# Load and preprocess data
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

assert X_train.shape == (60000, 28, 28)
assert X_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_train[0],)
plt.show()
#print(X_train[0])

In [ ]:
# Create the LeNet model
model = Sequential()

# Layer 1: Convolutional + Average Pooling
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(AveragePooling2D())

model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))# Layer 2: Convolutional + Average Pooling
model.add(AveragePooling2D())

model.add(Flatten())# Flatten the data for the fully connected layers
model.add(Dense(120, activation='relu'))# Layer 3: Fully Connected
model.add(Dense(84, activation='relu'))# Layer 4: Fully Connected
model.add(Dense(10, activation='softmax'))# Output Layer: Fully Connected with softmax activation
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train,to_categorical(y_train),epochs=10)

In [ ]:
# Prune the model
def prune_low_magnitude(model, pruning_ratio, threshold, sparsity_target):
    # Implement the pruning function here
    pruned_model = model
    return pruned_model

pruned_model = prune_low_magnitude(model, 0.5, 0.5, 0.8)

# Quantize the pruned model
def quantize_model(model):
    # Implement the quantization function here
    quantized_model = model
    return quantized_model

quantized_model = quantize_model(pruned_model)

# Define the DFO algorithm
def dfo(pruning_ratio, quantization_precision):
    # Prune the model based on the pruning ratio
    pruned_model = prune_low_magnitude(model, pruning_ratio, 0.5, 0.8)

    # Quantize the pruned model based on the quantization precision
    quantized_model = quantize_model(pruned_model)

    # Evaluate the performance of the pruned and quantized model
    loss, accuracy = quantized_model.evaluate(X_test, to_categorical(y_test))
    return accuracy

In [ ]:
# Run the DFO algorithm
best_accuracy = 0
best_pruning_ratio = 0
best_quantization_precision = 0

for i in range(10):
    pruning_ratio = np.random.uniform(0, 1)
    quantization_precision = np.random.uniform(0, 1)
    accuracy = dfo(pruning_ratio, quantization_precision)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_pruning_ratio = pruning_ratio
        best_quantization_precision = quantization_precision

print(f"Best pruning ratio: {best_pruning_ratio}")
print(f"Best quantization precision: {best_quantization_precision}")
print(f"Best accuracy: {best_accuracy}")

# DFO

In [ ]:
# Initialize parapeter for DFo
num_flies = 10
num_iterations = 10
pruning_ratio_bounds = (0,1)
quantization_precision_bounds = (0,1)

In [ ]:
# Initialize flies' positions randomly within the search space
flies = np.random.uniform([pruning_ratio_bounds[0], quantization_precision_bounds[0]], 
                          [pruning_ratio_bounds[1], quantization_precision_bounds[1]], 
                          (num_flies, 2))

In [ ]:
# Initialize best positions and accuracies for each fly
best_positions = flies.copy()
best_accuracies = np.zeros(num_flies)
global_best_position = flies[0]
global_best_accuracy = 0

In [ ]:
# Evaluate initial positions
for i in range(num_flies):
    accuracy = dfo(flies[i][0], flies[i][1])
    best_accuracies[i] = accuracy
    if accuracy > global_best_accuracy:
        global_best_accuracy = accuracy
        global_best_position = flies[i]

In [ ]:
# Main optimization loop
for iteration in range(num_iterations):
    for i in range(num_flies):
        # Dispersal: Randomly move some flies
        if np.random.rand() < 0.1:  # Dispersal probability
            flies[i] = np.random.uniform([pruning_ratio_bounds[0], quantization_precision_bounds[0]], 
                                         [pruning_ratio_bounds[1], quantization_precision_bounds[1]])
        # Attraction: Move flies towards the best-known positions
        else:
            flies[i] += 0.1 * (global_best_position - flies[i])  # Attraction to global best

        # Ensure flies stay within bounds
        flies[i] = np.clip(flies[i], [pruning_ratio_bounds[0], quantization_precision_bounds[0]], 
                           [pruning_ratio_bounds[1], quantization_precision_bounds[1]])

        # Evaluate new position
        accuracy = dfo(flies[i][0], flies[i][1])
        if accuracy > best_accuracies[i]:
            best_accuracies[i] = accuracy
            best_positions[i] = flies[i]
        if accuracy > global_best_accuracy:
            global_best_accuracy = accuracy
            global_best_position = flies[i]

print(f"Best pruning ratio: {global_best_position[0]}")
print(f"Best quantization precision: {global_best_position[1]}")
print(f"Best accuracy: {global_best_accuracy}")

In [ ]:
# Mock data for plotting (since we don't have the actual `model`, `X_test`, and `y_test`)
# Replace these with your actual data and model evaluation results.
num_flies = 10
num_iterations = 20

# Example data for accuracies over iterations for each fly (random data for illustration)
accuracies = np.random.rand(num_iterations, num_flies)
global_best_accuracies = np.max(accuracies, axis=1)

# Plotting the results
plt.figure(figsize=(12, 6))